In [1]:
# We need Qiskit, Pennylane and Abraxas
from qiskit.circuit.library import EfficientSU2
from abrax import toPrime, toPennylane
import pennylane as qml

In [2]:
# Let us initialise Qiskit SU2
qc = EfficientSU2(3, reps=1).decompose()
qc.draw()

┌──────────┐┌──────────┐                  ┌──────────┐ ┌──────────┐
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[3]) ├──────────■───────┤ Ry(θ[6]) ├─┤ Rz(θ[9]) ├
     ├──────────┤├──────────┤        ┌─┴─┐     ├──────────┤┌┴──────────┤
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[4]) ├──■─────┤ X ├─────┤ Ry(θ[7]) ├┤ Rz(θ[10]) ├
     ├──────────┤├──────────┤┌─┴─┐┌──┴───┴───┐┌┴──────────┤└───────────┘
q_2: ┤ Ry(θ[2]) ├┤ Rz(θ[5]) ├┤ X ├┤ Ry(θ[8]) ├┤ Rz(θ[11]) ├─────────────
     └──────────┘└──────────┘└───┘└──────────┘└───────────┘

In [3]:
# We use Abraxas String as a pseudo-IR
string = toPrime(qc)
print(string)

-0 ry(θ[0]) rz(θ[3]) cx(1)    ry(θ[6]) 
-1 ry(θ[1]) rz(θ[4]) cx(2)    ry(θ[7]) 
-2 ry(θ[2]) rz(θ[5]) ry(θ[8]) rz(θ[11])


In [4]:
# We can now use the 'maker' to run all the necessary gates
#  and pass in params into it
maker, params = toPennylane(string)
def circ(params):
  maker(qml, params)

  return qml.probs()

circuit = qml.QNode(circ, qml.device('default.qubit', wires=3))

In [5]:
# let's use 0.0, 0.1, 0.2...
params = [0.1 * i for i in range(len(params))]
circuit(params)

tensor([0.70065054, 0.05064247, 0.09717805, 0.03203952, 0.08211337,
        0.00629748, 0.02449996, 0.00657861], requires_grad=True)